# Climate Risk Premium Analysis - Samcheok Power Plant

This notebook runs the full CRP model across multiple scenarios and visualizes the results.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.pipeline.runner import CRPModelRunner
from src.reporting.plots import (
    plot_spreads, plot_cashflow_waterfall, plot_capacity_factor_trajectory,
    plot_npv_comparison
)

## 1. Run Multi-Scenario Analysis

In [ ]:
# Initialize runner
base_dir = Path.cwd().parent
runner = CRPModelRunner(base_dir)

# Run all default scenarios
results = runner.run_multi_scenario()

print(f"Ran {len(results)} scenarios:")
for name in results.keys():
    print(f"  - {name}")

## 2. Export Results to CSV

In [ ]:
# Export to processed folder
output_dir = base_dir / "data" / "processed"
paths = runner.export_results(results, output_dir)

print("\nExported files:")
for name, path in paths.items():
    print(f"  {name}: {path}")

## 3. Load and Display Summary Metrics

In [ ]:
# Load scenario comparison
metrics_df = pd.read_csv(output_dir / "scenario_comparison.csv")
metrics_df

## 4. Visualize NPV and IRR Comparison

In [ ]:
fig = plot_npv_comparison(metrics_df)
fig.show()

## 5. Visualize Climate Risk Premium

In [ ]:
# Filter to scenarios with CRP data
risk_scenarios = metrics_df[metrics_df['scenario'] != 'baseline'].copy()

if len(risk_scenarios) > 0:
    fig = plot_spreads(risk_scenarios)
    fig.show()
else:
    print("No risk scenarios with CRP data found")

## 6. Capacity Factor Trajectories

In [ ]:
# Load cashflow CSVs
cashflow_dfs = {}
for scenario_name in results.keys():
    cf_path = output_dir / f"cashflow_{scenario_name}.csv"
    if cf_path.exists():
        cashflow_dfs[scenario_name] = pd.read_csv(cf_path)

fig = plot_capacity_factor_trajectory(cashflow_dfs)
fig.show()

## 7. Cash Flow Waterfall (Baseline)

In [ ]:
if 'baseline' in cashflow_dfs:
    fig = plot_cashflow_waterfall(cashflow_dfs['baseline'], "Baseline")
    fig.show()

## 8. Key Findings Summary

In [ ]:
# Display key metrics
if 'baseline' in metrics_df['scenario'].values:
    baseline = metrics_df[metrics_df['scenario'] == 'baseline'].iloc[0]
    
    print("=" * 60)
    print("BASELINE SCENARIO")
    print("=" * 60)
    print(f"NPV: ${baseline['npv_million']:.1f} million")
    print(f"IRR: {baseline['irr_pct']:.2f}%")
    print(f"Avg DSCR: {baseline['avg_dscr']:.2f}")
    print(f"Min DSCR: {baseline['min_dscr']:.2f}")
    print(f"LLCR: {baseline['llcr']:.2f}")
    print()
    
    # Show worst-case scenario
    worst_case = risk_scenarios.loc[risk_scenarios['crp_bps'].idxmax()] if len(risk_scenarios) > 0 else None
    
    if worst_case is not None:
        print("=" * 60)
        print(f"WORST-CASE SCENARIO: {worst_case['scenario'].upper()}")
        print("=" * 60)
        print(f"NPV Loss: ${worst_case['npv_loss_million']:.1f} million")
        print(f"Expected Loss: {worst_case['expected_loss_pct']:.2f}%")
        print(f"Climate Risk Premium: {worst_case['crp_bps']:.0f} bps")
        print(f"WACC Increase: {worst_case['wacc_baseline_pct']:.2f}% → {worst_case['wacc_adjusted_pct']:.2f}%")
        print(f"Debt Spread: {worst_case['debt_spread_bps']:.0f} bps")